<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [6]</a>'.</span>

In [1]:
from edc import print_info
print_info("geodb-sharing")


***Notebook Title***  
GeoDB: Sharing Data

***Notebook Description***  
Euro Data Cube GeoDB: Sharing Data


***Notebook Dependencies***  
This notebook requires an active subscription to:
* EDC GeoDB


In [2]:
from edc import setup_environment_variables
setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:
* `GEODB_API_SERVER_PORT`, `GEODB_API_SERVER_URL`, `GEODB_AUTH_AUD`, `GEODB_AUTH_CLIENT_ID`, `GEODB_AUTH_CLIENT_SECRET`, `GEODB_AUTH_DOMAIN`

The following additional environment variables have been loaded from `~/custom.env`:
* `AWS_BUCKET`
* `DAPA_URL`
* `DB_HOST`, `DB_NAME`, `DB_PASSWORD`, `DB_USER`
* `OGC_EDC_URL`
* `REFERENCE_DATA`


In [3]:
from edc import check_compatibility
check_compatibility("user-0.19.6")

This notebook is compatible with this base image version (user-0.19.6).

## Sharing Data

In [4]:
from xcube_geodb.core.geodb import GeoDBClient

In [5]:
geodb = GeoDBClient()
geodb.whoami

'geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04'

### Publish a Collection to the World

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [6]:
geodb.list_my_grants()

AttributeError: 'GeoDBClient' object has no attribute 'list_my_grants'

In [ ]:
geodb.publish_collection("land_use")

### Accessing Collection as a different User

Please be aware that you cannot change the user easily within a Notebook. Here, we changed the credentials in the background. The following cells will not run and are for demonstration only.

In [ ]:
geodb = GeoDBClient()
geodb.whoami

In [ ]:
geodb.get_collection('land_use', database='geodb_admin')

### Revoke access

In [ ]:
geodb.list_my_grants()

In [ ]:
geodb.unpublish_collection("land_use")

In [ ]:
geodb.list_my_grants()

In [ ]:
gdf = geodb.get_collection_by_bbox(collection="land_use", database="geodb_admin", bbox=(452750.0, 88909.549, 464000.0, 102486.299),
                comparison_mode="contains", bbox_crs=3794, limit=2, offset=10)
gdf

### Accessing Collection as a different User

Please be aware that you cannot change the user easily within a Notebook. Here, we changed the credentials in the background. The following cells will not run and are for demonstration only.

In [ ]:
geodb = GeoDBClient()
geodb.whoami

In [ ]:
geodb.get_collection('land_use', database='geodb_admin')